<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Creating Features

### Lesson Objectives

* After completing this lesson, you should be able to:
  - Create features for your own datasets
  - Learn about and set relevant model parameters
  
### Overall Plan of Attack

* Make some of the data more useful by adding some columns to the dataset that re-encode a few fields
* Using groupBy on the Grant_Application_ID, aggregate/summarize the data about researchers
* When necessary, create StringINdexers for the categorical features

### Re-encode some data

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

spark = org.apache.spark.sql.SparkSession@6d4fe7b0


org.apache.spark.sql.SparkSession@6d4fe7b0

In [2]:
val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
  load("/resources/data/grantsPeople.csv")

data.show()

+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

data = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]


[Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]

In [3]:
val researchers = data.
  withColumn ("phd", data("With_PHD").equalTo("Yes").cast("Int")).
  withColumn ("CI", data("Role").equalTo("CHIEF_INVESTIGATOR").cast("Int")).
  withColumn("paperscore", data("A2") * 4 + data("A") * 3)
data.show()

+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

researchers = [Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]


[Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]

### Summarize Team Data

In [4]:
val grants = researchers.groupBy("Grant_Application_ID").agg(
  max("Grant_Status").as("Grant_Status"),
  max("Grant_Category_Code").as("Category_Code"),
  max("Contract_Value_Band").as("Value_Band"),
  sum("phd").as("PHDs"),
  when(max(expr("paperscore * CI")).isNull, 0).
    otherwise(max(expr("paperscore * CI"))).as("paperscore"),
  count("*").as("teamsize"),
  when(sum("Number_of_Successful_Grant").isNull, 0).
    otherwise(sum("Number_of_Successful_Grant")).as("successes"),
  when(sum("Number_of_Unsuccessful_Grant").isNull, 0).
    otherwise(sum("Number_of_Unsuccessful_Grant")).as("failures")
)

grants.show()

+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|Grant_Application_ID|Grant_Status|Category_Code|          Value_Band|PHDs|paperscore|teamsize|successes|failures|
+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|                 148|           0|  GrantCat30B|ContractValueBandUnk|null|         6|       1|        0|       1|
|                 463|           1|  GrantCat30C|ContractValueBandUnk|null|         0|       1|        1|       0|
|                 471|           0|  GrantCat30B|  ContractValueBandA|   1|       127|       2|        1|       5|
|                 496|           0|  GrantCat30B|  ContractValueBandA|null|         0|       1|        1|       3|
|                 833|           1|  GrantCat10A|  ContractValueBandF|null|         0|       1|        0|       0|
|                1088|           1|  GrantCat50A|  ContractValueBandA|   1|     

grants = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]


[Grant_Application_ID: int, Grant_Status: int ... 7 more fields]

### Handle Categorical Features

In [ ]:

import org.apache.spark.ml.feature.StringIndexer

val value_band_indexer = new StringIndexer().
  setInputCol("Value_Band").
  setOutputCol("Value_index").
  fit(grants)
  
val category_indexer = new StringIndexer().
  setInputCol("Category_Code").
  setOutputCol("Category_index").
  fit(grants)
  
val label_indexer = new StringIndexer().
  setInputCol("Grant_Status").
  setOutputCol("status").
  fit(grants)


### Gather the Features into a Vector

In [ ]:

import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
  setInputCols(Array(
    "Value_index"
    ,"Category_index"
    ,"PHDs"
    ,"paperscore"
    ,"teamsize"
    ,"successes"
    ,"failures"
  )).
  setOutputCol("assembled")
  

### Setup a Classifier

In [ ]:

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rf = new RandomForestClassifier().
  setFeaturesCol("assembled").
  setLabelCol("status").
  setSeed(42)

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.